# Procesos en phyton

Antes de empezar es importante indicar un par de detalles relacionados con el entorno. El multiproceso en *jupyter* lo que hace es lanzar un nuevo núcleo de *jupyter* con el proceso asociado. Como ya se ha explicado en la teoría si el padre no hace un wait y el proceso hijo ha acabado se queda en estado de zombie. En el caso de *jupyter* el proceso estará zombie pero en núcleo de python ejecutado continua ejecutándose y ya no sólo ocupa memoria sino que también consume tiempo de procesamiento.

En la mayoría de los casos basta con pulsar repetidamente el botón de pausa pero como es posible que que a veces no se solucione se aportan dos herramientas. La primera es para ver cuántos procesos están ejecutándose y, por tanto, probablemente zombies (salvo uno). Para ello basta con ejecutar la siguiente intrucción si estamos en Linux.

In [ ]:
!echo "\033[31;01mHay "`ps aux | grep ipykernel | grep -v grep | wc | awk '{print $1}'`" procesos ejecutándose\033[00m"

El problema es que es casi imposible determinar qué núcleos de *jupyter* se corresponden con los procesos zombie y cuales no por tanto es encesario matar todos y se reiniciará este mismo *jupyter*. Esto no implica ningún problema salvo que se olvidan las ejecuciones de celdas anteriores y es necesario volver a empezar las ejecuciones. Para eliminar estos procesos podemos ejecutar el siguiente comando de linux.

In [ ]:
!kill `ps aux | grep ipykernel | grep -v grep | awk '{print $2}'`

Una vez entendido cómo funcionan los procesos vamos a ver cómo funcionan los procesos en python

Lo primero que necesitamos es importar las funciones para trabajar con elementos del sistema operativo. Para ello hacemos:

In [ ]:
import os

También importamos time para poder hacer sleep y la función randint de random para poder generar números aleatorios

In [ ]:
import time
import random as rnd

Una vez hecho vamos a crear un ejemplo:

In [ ]:
def parent():
    for i in range(10):
        newpid = os.fork()
        if newpid == 0:
            time.sleep(rnd.randint(0,9))
            print("Soy el hijo %d" % os.getpid())
            os._exit(1)
            break
        else:
            pids = (os.getpid(), newpid)
            print("padre: %d, hijo: %d" % pids)
            time.sleep(rnd.randint(0,3))
    else:
        pass
    
parent()

El último else es se debe a una característica propia del python que no viene a cuento contar ahora mismo. Sin embargo lo que sí es importante es que los hojos no han acabado pero no han sido recogidos y, por tanto están zombies. Vamos a recogerlos del sistema operativo:

In [ ]:
    print("Retornos:")
    for i in range(10): 
        a = os.wait()
        print(a)

## Organizando un poco

Vamos a crear nuestras propias funciones Fork, Wait, Exit y RandSleep con el fin de simplificar la resolución de problemas. 

- El procedimiento Fork devuelve 0 o el pid del hijo.
- El procedimiento Wait convertirá la tupla con dos elementos de _os.\_wait_ en una tupla con tres elementos. El primero es el _pid_, el segundo el valor de retorno y el tercero el número de la señal que ha provocado la finalización del proceso.
- El proceso Exit recibe el valor que devolverá al padre.
- El proceso RandSleep puede recibir un parámetro que es el valor máximo de segundos que puede esperar y si no se pone el parámetro su valor por defecto es 5s.

In [2]:
from os import fork, _exit, wait, getpid, getppid
from time import sleep
from random import randint

def RandSleep(max=5):
    sleep(randint(0,max))

def Fork():
    pid = fork()
    return pid

def Exit(a):
    _exit(a)
    
def Wait():
    b = ()
    try:
        a = wait()
    except:
        return -1
    b = (a[0], a[1]/256, a[1]%256)
    return b

## Aprendiendo a crear procesos

Vamos a hacer una prueba sencilla

In [ ]:
for i in range(4):
    pid = Fork()
    if pid:
        print("Soy %d y he creado el hijo %d" % (getpid(),pid))
    else:
        print("Soy el hijo %d y mi padre es el %d" % (getpid(),getppid()))
        Exit(0)
          
for i in range(4):
    retv = Wait()
    print("He acabado. Era el proceso %d y he acabado con valor %d debido a la señal %d" % retv)
    


Ahora vamos a hacer algo más complejo. Vamos a resolver un pequeño problema. Queremos cada hijo lance un hijo hasta un valor dado de profundidad.

In [3]:
profundidad = -1
profundidad_max = 27

def proceso():
    global profundidad, profundidad_max
    
    profundidad += 1
    
    if profundidad<profundidad_max:
        if Fork():
            print("Soy %d" % getpid() + ", mi hijo %d ha devuelto %d con la señal %d" % Wait())
        else:
            proceso()
            Exit(profundidad)
    
proceso()

Soy 2964, mi hijo 2965 ha devuelto 27 con la señal 0
Soy 2963, mi hijo 2964 ha devuelto 26 con la señal 0
Soy 2962, mi hijo 2963 ha devuelto 25 con la señal 0
Soy 2961, mi hijo 2962 ha devuelto 24 con la señal 0
Soy 2960, mi hijo 2961 ha devuelto 23 con la señal 0
Soy 2959, mi hijo 2960 ha devuelto 22 con la señal 0
Soy 2958, mi hijo 2959 ha devuelto 21 con la señal 0
Soy 2957, mi hijo 2958 ha devuelto 20 con la señal 0
Soy 2956, mi hijo 2957 ha devuelto 19 con la señal 0
Soy 2955, mi hijo 2956 ha devuelto 18 con la señal 0
Soy 2954, mi hijo 2955 ha devuelto 17 con la señal 0
Soy 2953, mi hijo 2954 ha devuelto 16 con la señal 0
Soy 2952, mi hijo 2953 ha devuelto 15 con la señal 0
Soy 2951, mi hijo 2952 ha devuelto 14 con la señal 0
Soy 2950, mi hijo 2951 ha devuelto 13 con la señal 0
Soy 2949, mi hijo 2950 ha devuelto 12 con la señal 0
Soy 2948, mi hijo 2949 ha devuelto 11 con la señal 0
Soy 2947, mi hijo 2948 ha devuelto 10 con la señal 0
Soy 2946, mi hijo 2947 ha devuelto 9 con la se

Como todos los procesos esperan a su hijo para retornar todos finalizan en orden inverso a cómo han sido creados.

#### Pregunta:
¿Por qué a pesar de ser creados de forma ordenada el menasje del padre aparece después del de muchos hijos?
#### Ejercicio:
Representa el árbol de procesos.

## Algo más difícil

Vamos a modificar el ejemplo para que cada proceso, incluido el padre lance dos procesos y espere a que terminen los dos.

In [ ]:
profundidad = -1
profundidad_max = 3
    
def proceso():
    global profundidad, profundidad_max
    
    profundidad += 1
    
    if profundidad<profundidad_max:
        if Fork():
            if Fork():
                print("Soy %d" % getpid() + " Mi hijo %d ha devuelto %d con la señal %d" % Wait())
                print("Soy %d" % getpid() + " Mi hijo %d ha devuelto %d con la señal %d" % Wait())
            else:
                proceso()
                Exit(profundidad)
        else:
            proceso()
            Exit(profundidad)
    
proceso()
print("Y yo soy el padre %s" % getpid())

## Problemas

### Problema 1
Dado el siguiente código en pseudoC crea un programa en python para comprobarlo partiendo de los anteriores ejemplos. Como no hay ningún wait es evidente que todos los procesos menos el padre quedan zombies. 

- Dibuja el árbol de procesos
- Añade los waits necesarios para que no haya zombies
- Muestra la salida que genera la ejecucion de este código. ¿Puede producirse otra salida?  Justifica la respuesta y si es necesario añade esperas aleatorias en distintos lugares para comprobar si la salida es determinista o si no lo es.

***

```
main()
{
    if (fork()){
        if (fork()){
            printf("A\n");
        } else {
            printf("B\n");
        }
    } else
        printf("C\n");
    exit(0);
}
```
***

El código equivalente en python sin añadir nada de lo solicitado y utilizando los procedimientos definidos anteriormente está en la siguiente celda. Comprueba antes de usarlo que el código es equivalente. Añade los elementos solicitados y ejecuta el código.

In [ ]:
if Fork():
    if Fork():
        print("A")
    else:
        print("B")
        Exit(2)
else:
    print("C")

### Problema 2

A partir del siguiente código realiza las siguientes acciones:

- Dibuja el diagrama de procesos identificando cada proceso con una letra
- Predice la salida del código e indica si es determinista.
- Crea un código equivalente en python.
- Añade los wait necesarios para que no haya procesos huérfanos.
- Añade el código necesario necesario para que los respectivos procesos padre sean los últimos en ejecutarse.

***
```
main()
{  
    int i;
    for (i=0; i<3; i++)
        if (fork()){
            printf("i=%d, soy padre\n",i);
            exit(0);
        } else {
            printf("i=%d, soy hijo\n",i);
            exit(0);
        }
}
```
***

### Problema 3

Cuántos procesos creamos con este código ¿Por qué no funciona correctamente? Pruébalo e investiga por qué sucede.

In [ ]:
i=0

def principal():
    global i
    i += 1
    if i < 4:
        if Fork():
            principal()
        else:
            principal()
            Exit(i)

principal()
for i in range(7):
    Wait()


## ¿Qué pasa con las variables?

Como sabemos a partir de la teoría, cuando se crea un proceso el SO copia toda la imagen del proceso para el proceso hijo. En esta situación, a partir del instante de la copia cada uno de los procesos tiene su propia variable. Sólo en el instante en el momento de crear el proceso tanto el padre como el hijo tienen el mismo valor de la variable puesto que el hijo es una copia del padre, incluidos los datos, en el instante en el que se crea el hijo.

Veamos varios ejemplos.

### Usando variables globales

imaginemos el siguiente código en pseudoC:

***
```
int a=0;
max = 7

main()
{
    if (a < 7)
    {
        ++a;
        if(!fork())
        {
            main();
            print ("%d\n",a);
            a += 10;
            exit(a);
        }
        else exit(a);
    }
    else wait();
}

```
***

Implementado en python es de la siguiente forma:

In [ ]:
a = 0
max = 7

def proceso():
    global a
    
    if a < 7:
        a += 1
        if Fork() == 0:
            proceso()
            print(a)
            a += 10
            Exit(a)
        else:
            Wait()

proceso()
print(a)

##### Problema
Dibuja el esquema de procesos y explica el resultado.

# Substituyendo procesos

Ya que sabemos bien lo que es un proceso vamos a ver cómo substituir un proceso por otro en tiempo de ejecución. ¿Para qué sirve esto? Pues por ejemplo, una shell de bash es un proceso y cuando nosotros ejecutamos una instrucción realmente lo que se hace es lanzar otro proceso shell que se verá substituido por la instrucción correspondiente. Existe un conjunto de instrucciones que realizan este proceso:

- execl(path, arg0, arg1, ...)
- execle(path, arg0, arg1, ..., env)
- execlp(file, arg0, arg1, ...)
- execlpe(file, arg0, arg1, ..., env)
- execv(path, args)
- execve(path, args, env)
- execvp(file, args)
- execvpe(file, args, env)

In [ ]:
from os import execl, execle, execlp, execlpe, execv, execve, execvp, execvpe

args = ["-la"]

if Fork() == 0: 
    execvp("ls",args)
if Fork() == 0: 
    execlp("ls","-la")
if Fork() == 0: 
    execlp("ps","aux")

Como se puede ver no se obtiene salida alguna. Esto es porque estos comandos tienen su propio dispositivo de salida que es stdout y no el interfaz de jupyter. El stdout asociado es la shell donde se haya ejecutado jupyter. Se puede observar ahí la salida.

## Problemas de las hojas de problemas, cómo probarlos

Vamos a modificar Wait para que nos de imformación por pantalla cuando el Wait lo realice un proceso sin hijos. También modificamos Fork y Exit para que nos aporten información.

In [14]:
from os import fork, _exit, wait, getpid, getppid
from time import sleep
from random import randint

def RandSleep(max=5):
    sleep(randint(0,max))

def Exit(a):
    print("Soy el proceso %d y he terminado con retorno %d" % (getpid(),a))
    _exit(a)

def Fork():
    pid = fork()
    if pid:
        print ("Soy %d y soy el padre de %d" % (getpid(),pid))
    return pid


def Wait():
    b = ()
    try:
        a = wait()
        b = (a[0], a[1]/256, a[1]%256)
        print("El proceso %d" % getpid() + " ha realizado un Wait y ha encontrado al hijo %d retornando %d con el código de estatus %d" % b)
    except:
        print("El proceso %d ha intentado hacer un Wait y no tiene hijos" % getpid())
        return False
    return b

Una pequeña prueba de los cambios

In [ ]:
if Fork() > 0:
    Wait()
else:
    Exit(2)

In [ ]:
"""
main()
{
    int i=0;
    pid_t pid;

    while(i < 2)
    {
        pid = fork();
        if(pid) fork(); 
        else wait();
        ++i;
    }
    if(pid) wait();
    wait();
}
"""

i = 0
while i < 2:
    pid = Fork()
    if pid: Fork()
    else: Wait()
    i += 1
if pid: Wait()


Soy 2852 y soy el padre de 3341
Soy 2852 y soy el padre de 3342
El proceso 3343 ha intentado hacer un Wait y no tiene hijos
Soy 2852 y soy el padre de 3341
Soy 3342 y soy el padre de 3347
Soy 2852 y soy el padre de 3341
Soy 2852 y soy el padre de 3342
Soy 2852 y soy el padre de 3343
El proceso 3344 ha intentado hacer un Wait y no tiene hijos
El proceso 3341 ha intentado hacer un Wait y no tiene hijos
Soy 3342 y soy el padre de 3356
Soy 2852 y soy el padre de 3341
El proceso 3347 ha intentado hacer un Wait y no tiene hijos
Soy 3341 y soy el padre de 3359
El proceso 3356 ha intentado hacer un Wait y no tiene hijos
El proceso 3359 ha intentado hacer un Wait y no tiene hijos
Soy 3341 y soy el padre de 3366
Soy 2852 y soy el padre de 3341
Soy 2852 y soy el padre de 3342
Soy 2852 y soy el padre de 3343
Soy 2852 y soy el padre de 3344
El proceso 3366 ha intentado hacer un Wait y no tiene hijos


In [ ]:
"""
main()
{
    int i=0;

    while(!fork())
    {
        if(fork()) wait();
        wait();
        if(i == 1) break;
        else i++;
    }
}
"""

i=0

while Fork() == 0:
    if Fork(): Wait()
    Wait()
    if i == 1: 
        print ("Prueba")
        break
    else: i = i+1

In [ ]:
"""
main(){
    char d = 'A';

    for( i = 0 ; i < 2 ; ++i){
        ++d;
        if (fork()){
            ++d;
            wait();
            if(fork()){
                ++d;
                wait();
            } else printf("Soy %c\n",d); 
        } else printf("Soy %c\n",d); 
    }
}
"""

d = 'A'

for i in range(2):
    d = chr (ord(d)+1)
    if Fork():
        d = chr (ord(d)+1)
        Wait()
        if Fork():
            Wait()
        else: 
            print("FI: Soy %c, mi pid es %d y mi padre es %d" % (d,getpid(), getppid()))
    else: 
        print("FE: Soy %c, mi pid es %d y mi padre es %d" % (d,getpid(), getppid()))